In [1]:
import numpy as np
import re
import math

In [2]:
test_input = """Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."""

In [115]:
import requests

URL = "https://adventofcode.com/{year}/day/{day}/input"
url = URL.format(year='2022', day=19)
cookies = {"session":"53616c7465645f5fd345d8c12076a357c8d31b2c0c4ad1c152eefbd5c1cf09e0c8d6d7f657fc558887d443d2d07fbc90728ada9b6a91afcecb9926ffb4981a93"}
req = requests.get(url, cookies=cookies)

if req.status_code != 200:
    print("Error. Got status:", req.status_code)
    print(req.text)
    exit(3)
else:
    my_input = req.text

In [120]:
def get_blueprints(this_input):
    blueprints = []
    for l in this_input.split("\n"):
        if l == "" :
            continue
        m = [int(i) for i in re.findall(" \d* ",l)]
        blueprints.append(
            [
                [m[0],0,0,0],
                [m[1],0,0,0],
                [m[2],m[3],0,0],
                [m[4],0,m[5],0]
            ]
        )
    return blueprints

In [121]:
def one_digging_step(R, M):
    return R, [M[i] + R[i] for i in range(4)]

def construction_option_withNone(R, M, b) :
    opt = [None]
    for i in range(4):
        if all([b[i][ii]<=M[ii] for ii in range(4)]) :
            opt.append(i)
    return opt

def construction_option_canBeEmpty(R, M, b) :
    opt = []
    for i in range(4):
        if all([b[i][ii]<=M[ii] for ii in range(4)]) :
            opt.append(i)
    return opt

def buildR(R, M, b, iR):
    Rn = R.copy()
    Mn = M.copy()
    if iR is not None :
        Rn[iR] += 1
        Mn = [Mn[i]-b[iR][i] for i in range(4)]
    return Rn, Mn

In [122]:
def time_to_build_R0(R,M,b) :
    T0 = math.ceil ( (b[0][0]-M[0]) / R[0] ) + 1 
    return T0,0

def time_to_build_R1(R,M,b,T0) :
    # if no new robot :
    T1asis = math.ceil(  (b[1][0]-M[0])/R[0] ) + 1

    if T0 < T1asis :
        # maybe better to build one more R0 :
        T1more0 = math.ceil( (b[1][0]+T0-M[0]+b[0][0])/(R[0]+1) ) + 1
        if T1more0 <= T1asis :
            return max(2,T1more0),0
    return T1asis,1

def time_to_build_R2(R,M,b,T0,T1) :
    
    if R[1] == 0 :
        return math.inf, 1
    
    # if no new robot :
    T2asis = max( math.ceil( (b[2][0]-M[0])/R[0] ) , math.ceil( (b[2][1]-M[1])/R[1] ) ) + 1
    
    if T1 < T2asis :
        # if one more R1
        # b[2][0] = M[0] + T * R[0] - b[1][0]
        # b[2][1] = M[1] + T * R[1] + (T-T1) 
        T2more1 = max( math.ceil( (b[2][0]+b[1][0]-M[0])/R[0] ) , math.ceil( (b[2][1]+T1-M[1])/(R[1]+1) ) ) + 1
    else :
        T2more1 = math.inf
        
    if T0 < T2asis :
        # if one more R0
        # b[2][0] = M[0] + T * R[0] - b[0][0] + (T-T0) 
        T2more0 = max( math.ceil( (b[2][0]+T0-M[0]+b[0][0])/(R[0]+1) ) , math.ceil( (b[2][1]-M[1])/R[1] ) ) + 1
    else :
        T2more0 = math.inf
    
    T2 = min(T2asis,T2more1,T2more0)
    
    if T2more1 == T2 :
        return max(2,T2), 1
    if T2more0 == T2 :
        return max(2,T2), 0
    else :
        return T2, 2


def time_to_build_R3(R,M,b,T0,T1,T2) :
    
    if R[2] == 0 :
        return math.inf, 2
    
    # if no new machine :
    T3asis = max( math.ceil( (b[3][0]-M[0])/R[0] ) , math.ceil( (b[3][2]-M[2])/R[2] ) ) + 1
    
    if T0 < T3asis :
        # if one more R0
        # b[3][0] = M[0] + T*R[0] - b[0][0] + (T-T0)   ==>   T = (b[3][0]-M[0]+b[0][0]+T0)/(R[0]+1)
        # b[3][2] = M[2] + T*R[2]
        T3more0 = max(math.ceil( (b[3][0]-M[0]+b[0][0]+T0)/(R[0]+1) ) ,
                      math.ceil( (b[3][2]-M[2])/R[2] ) ) + 1
    else :
        T3more0 = math.inf
    
    if T2 < T3asis :
    # if one more R2
        # b[3][0] = M[0] + T*R[0] - b[2][0] ==> T = (b[3][0]-M[0]+b[2][0])/R[0]
        # b[3][2] = M[2] + T*R[2] + (T-T2)*1 ==> T = (b[3][2]-M[2]+T2)/(R[2]+1)
        T3more2 = max(math.ceil( (b[3][0]-M[0]+b[2][0])/R[0] ) ,
                      math.ceil( (b[3][2]-M[2]+T2)/(R[2]+1) ) ) + 1
    else :
        T3more2 = math.inf

    
    T3 = min(T3asis,T3more2,T3more0)
    
    if T3more2 == T3 :
        return max(2,T3), 2
    if T3more0 == T3 :
        return max(2,T3), 0
    else :
        return T3, 3

In [123]:
def time_to_build_R0_asis(R,M,b) :
    if M[0] >= b[0][0] :
        return 1
    return math.ceil(  (b[0][0]-M[0])/R[0] ) +1

def time_to_build_R1_asis(R,M,b) :
    if M[0] >= b[1][0] :
        return 1
    return math.ceil(  (b[1][0]-M[0])/R[0] ) +1

def time_to_build_R2_asis(R,M,b) :
    if R[1] == 0 :
        return math.inf
    return max( math.ceil( (b[2][0]-M[0])/R[0] ) , math.ceil( (b[2][1]-M[1])/R[1] ) , 0 ) + 1

def time_to_build_R3_asis(R,M,b) :
    if R[2] == 0 :
        return math.inf
    # if no new machine :
    return max( math.ceil( (b[3][0]-M[0])/R[0] ) , math.ceil( (b[3][2]-M[2])/R[2] ) , 0 ) + 1

def update_gather_and_build(R,M,b,iRbuilt,Ttobuilt) :
    Rn = R.copy()
    Mn = [M[i] + Ttobuilt * R[i] - b[iRbuilt][i] for i in range(4)]
    Rn[iRbuilt] += 1
    return Rn, Mn

def update_gather_till_end(R,M,b,currentT,TMAX) :
    return R, [M[i] + (TMAX-currentT) * R[i] for i in range(4)]

times_to_build_Rs = [time_to_build_R0_asis,time_to_build_R1_asis,time_to_build_R2_asis,time_to_build_R3_asis]

In [124]:

def get_one_path_shortest_time(b,TMAX) :
    
    t = 0
    R = [1,0,0,0]
    M = [0,0,0,0]
    
    while t < TMAX :

        T0,iR0 = time_to_build_R0(R,M,b)
        T1,iR1 = time_to_build_R1(R,M,b,T0)
        T2,iR2 = time_to_build_R2(R,M,b,T0,T1)
        T3,iR3 = time_to_build_R3(R,M,b,T0,T1,T2)

        Ts = [T0,T1,T2,T3]
        iR = [iR0,iR1,iR2,iR3]
        ir = 3

        while iR[ir] != ir :
            ir = iR[ir]

        for i in range(Ts[ir]-1) :
            
            R,M = one_digging_step(R, M)
            t += 1
            if t == TMAX :
                return M[3]


        R,M = one_digging_step(R, M)
        R,M = buildR(R, M, b, ir)
        
        t+=1
        
        if t == TMAX :
            return M[3]



def get_one_path_most_build(b,TMAX) :
    
    R = [1,0,0,0]
    M = [0,0,0,0]
    
    for t in range(TMAX) :
        
        opt = construction_option_canBeEmpty(R, M, b)
        
        R,M = one_digging_step(R, M)
        if len(opt) > 0:
            R,M = buildR(R, M, b, max(opt))

    return M[3]



In [125]:
def value(R,M,b,t) :
    
    V0 = b[0][0]
    V1 = b[1][0] 
    V2 = b[2][0] + b[2][1] * V1
    V3 = b[3][0] + b[3][2] * V2
    
    return (M[0] + M[1]*V1 + M[2]*V2 + M[3]*V3) + (R[0] + R[1]*V1 + R[2]*V2 + R[3]*V3) * (24-t) 

def min_collect(R,M,b,t,TMAX) :
    # factory breaks, it can only create one robot if it has not been created yet
    # [or the last level of robot : if not efficient enough]
    Mm = M.copy()
    Rm = R.copy()
    T = t
    # create robot if necessary
    if Rm[1] == 0 :
        T1 = time_to_build_R1_asis(R,M,b)
        Mm[0] = Mm[0] + T1*Rm[0] - b[1][0]
        Rm[1] = 1
        T += T1
        if T >= TMAX :
            return Mm[3]
    if Rm[2] == 0 :
        T2 = time_to_build_R2_asis(R,M,b)
        Mm[0] = Mm[0] + T2*Rm[0] - b[2][0]
        Mm[1] = Mm[1] + T2*Rm[1] - b[2][1]
        Rm[2] = 1
        T += T2
        if T >= TMAX :
            return Mm[3]
    if Rm[3] == 0 :
        T3 = time_to_build_R3_asis(R,M,b)
        Mm[0] = Mm[0] + T3*Rm[0] - b[3][0]
        Mm[2] = Mm[2] + T3*Rm[2] - b[3][2]
        Rm[3] = 1
        T += T3
        if T >= TMAX :
            return Mm[3]
    
    return Mm[3] + Rm[3]*(TMAX-T)


def max_collect(R,M,b,t,TMAX) :
    # robots are so effective that I have materials to build instantaneously
    T = t
    if R[1] == 0 :
        # first need to build machine 1
        T += 1
        if T >= TMAX :
            return M[3]
    if R[2] == 0 :
        # also need to build machine 2
        T += 1
        if T >= TMAX :
            return M[3]
    if R[3] == 0 :
        # and of course the last machine
        T +=1
        return M[3] + (TMAX-T) *(TMAX-T+1) / 2
    return M[3] + (TMAX-T)*( R[3]-1 + ((TMAX-T+1) / 2))

In [126]:
def update_list(li,t):
    return [li[i] for i in range(len(li)) if t[i]]

In [8]:
blueprints = get_blueprints(test_input)

In [60]:
ib = 1

b = blueprints[ib]
b

[[2, 0, 0, 0], [3, 0, 0, 0], [3, 8, 0, 0], [3, 0, 12, 0]]

In [107]:
all_R = [[1,0,0,0]]
all_M = [[0,0,0,0]]
all_T = [0]

# all_min = [min_collect(all_R[0],all_M[0],b,all_T[0])]
all_max = [max_collect(all_R[0],all_M[0],b,all_T[0])]

Overall_Min = max(min_collect(all_R[0],all_M[0],b,all_T[0]), get_one_path_shortest_time(b), get_one_path_most_build(b))
# if one_max < Overall_Min : then suppress
print("at start, min is :",Overall_Min)

at start, min is : 10


In [105]:
count = 0
while len(all_R) > 0 :
    
    R = all_R.pop(-1)
    M = all_M.pop(-1)
    T = all_T.pop(-1)
#     mi = all_min.pop(-1)
    ma = all_max.pop(-1)
    
    for iR in range(4) :
        
        TR = times_to_build_Rs[iR](R,M,b)
    
        if TR == math.inf :
            continue
        
        if T+TR >= 24 :
            # only gather and stop
            Rn, Mn = update_gather_till_end(R,M,b,T)
#             print("\t",Mn[3])
            Overall_Min = max(Overall_Min,Mn[3])
            # keep only those that might do better
            keep = [all_max[i]>Overall_Min for i in range(len(all_max))]
            all_R = update_list(all_R,keep)
            all_M = update_list(all_M,keep)
            all_T = update_list(all_T,keep)
            all_max = update_list(all_max,keep)
            
            continue

        Rn, Mn = update_gather_and_build(R,M,b,iR,TR)
        # update_gather_and_build(R,M,b,iRbuilt,Ttobuilt)
        Tn = T + TR
        man = max_collect(Rn,Mn,b,Tn)
        if man > Overall_Min :
            if Rn in all_R :
                all_idx = [i for i in range(len(all_R)) if all_R[i] == Rn and all_M[i] == Mn]
                if len(all_idx) > 0 :
                    if all_T[all_idx[0]] > Tn :
                        all_T[all_idx[0]] = Tn
                    continue
            all_R.append(Rn)
            all_M.append(Mn)
            all_T.append(Tn)
            all_max.append(man)
        
        if min(Mn) < 0:
            print("Problem : ", iR, TR)
            print("\t",R,M,T)
            print("\t",Rn,Mn,Tn)
            
    
    
    count += 1
    if count % 1000 == 0 :
        print(len(all_R),min(all_max),max(all_max),max(all_T))

14 15.0 171.0 18
11 15.0 171.0 18
9 21.0 171.0 17
13 15.0 171.0 19
13 15.0 171.0 18
9 15.0 171.0 18
11 15.0 171.0 18
13 15.0 171.0 18
7 15.0 171.0 17
7 36.0 171.0 15
10 15.0 171.0 18
12 15.0 171.0 18
10 15.0 171.0 18
5 15.0 171.0 18
6 15.0 171.0 17
7 15.0 171.0 18
14 15.0 171.0 18
12 15.0 171.0 18
12 15.0 171.0 18
11 21.0 171.0 17
10 21.0 171.0 17
12 15.0 171.0 18
8 15.0 171.0 18
12 15.0 171.0 18
11 15.0 171.0 18
11 15.0 171.0 18
11 15.0 171.0 18
8 15.0 171.0 18
9 21.0 171.0 17
8 15.0 171.0 17
8 28.0 171.0 16
8 15.0 171.0 18
6 21.0 171.0 17
10 15.0 171.0 18
7 21.0 171.0 17
12 15.0 136.0 18
13 15.0 136.0 18
17 11.0 136.0 21
18 11.0 136.0 20
14 11.0 136.0 20
15 11.0 136.0 20
14 11.0 136.0 20
12 21.0 136.0 17
14 15.0 136.0 18
9 15.0 136.0 17
16 11.0 136.0 20
15 11.0 136.0 20
15 15.0 136.0 18
11 15.0 136.0 18
14 15.0 136.0 19
16 11.0 136.0 21
15 11.0 136.0 21
12 15.0 136.0 18
15 11.0 136.0 20
12 15.0 136.0 18
14 11.0 136.0 20
16 15.0 136.0 19
11 21.0 136.0 17
14 15.0 136.0 18
7 28.0 136.0 

In [106]:
Overall_Min

12

In [117]:
blueprints = get_blueprints(my_input)

In [118]:

index_blueprint = []
result_blueprint = []

for ib in range(len(blueprints)) :

    b = blueprints[ib]


    all_R = [[1,0,0,0]]
    all_M = [[0,0,0,0]]
    all_T = [0]

    # all_min = [min_collect(all_R[0],all_M[0],b,all_T[0])]
    all_max = [max_collect(all_R[0],all_M[0],b,all_T[0])]

    Overall_Min = max(min_collect(all_R[0],all_M[0],b,all_T[0]), get_one_path_shortest_time(b), get_one_path_most_build(b))
    # if one_max < Overall_Min : then suppress
    print("\nFor blueprint",ib,", at start, min is :",Overall_Min)

    count = 0
    while len(all_R) > 0 :

        R = all_R.pop(-1)
        M = all_M.pop(-1)
        T = all_T.pop(-1)
    #     mi = all_min.pop(-1)
        ma = all_max.pop(-1)

        for iR in range(4) :

            TR = times_to_build_Rs[iR](R,M,b)

            if TR == math.inf :
                continue

            if T+TR >= 24 :
                # only gather and stop
                Rn, Mn = update_gather_till_end(R,M,b,T)
    #             print("\t",Mn[3])
                Overall_Min = max(Overall_Min,Mn[3])
                # keep only those that might do better
                keep = [all_max[i]>Overall_Min for i in range(len(all_max))]
                all_R = update_list(all_R,keep)
                all_M = update_list(all_M,keep)
                all_T = update_list(all_T,keep)
                all_max = update_list(all_max,keep)

                continue

            Rn, Mn = update_gather_and_build(R,M,b,iR,TR)
            # update_gather_and_build(R,M,b,iRbuilt,Ttobuilt)
            Tn = T + TR
            man = max_collect(Rn,Mn,b,Tn)
            if man > Overall_Min :
                if Rn in all_R :
                    all_idx = [i for i in range(len(all_R)) if all_R[i] == Rn and all_M[i] == Mn]
                    if len(all_idx) > 0 :
                        if all_T[all_idx[0]] > Tn :
                            all_T[all_idx[0]] = Tn
                        continue
                all_R.append(Rn)
                all_M.append(Mn)
                all_T.append(Tn)
                all_max.append(man)

            if min(Mn) < 0:
                print("Problem : ", iR, TR)
                print("\t",R,M,T)
                print("\t",Rn,Mn,Tn)



        count += 1
        if count % 100000 == 0 :
            print("at count",count,":")
            print("\tnumber of cases registered :" ,len(all_R))
            print("\tall_max from",min(all_max),"to",max(all_max))
            print("\tmost advanced time :",max(all_T))
            print("\tcurrent best min :",Overall_Min)
            
    
    index_blueprint.append(ib+1)
    result_blueprint.append(Overall_Min)
    print("FOr blueprint",ib,", at the end, opened",Overall_Min,"geodes.\n")
    
print(sum([index_blueprint[i]*result_blueprint[i] for i in range(len(result_blueprint))]))


For blueprint 0 , at start, min is : 3

For blueprint 1 , at start, min is : 0

For blueprint 2 , at start, min is : 0

For blueprint 3 , at start, min is : 0

For blueprint 4 , at start, min is : 9

For blueprint 5 , at start, min is : 0

For blueprint 6 , at start, min is : 4

For blueprint 7 , at start, min is : 2
at count 100000 :
	number of cases registered : 10
	all_max from 10.0 to 136.0
	most advanced time : 19
	current best min : 2
at count 200000 :
	number of cases registered : 6
	all_max from 6.0 to 136.0
	most advanced time : 20
	current best min : 2
at count 300000 :
	number of cases registered : 12
	all_max from 6.0 to 136.0
	most advanced time : 20
	current best min : 2
at count 400000 :
	number of cases registered : 14
	all_max from 3.0 to 120.0
	most advanced time : 21
	current best min : 2
at count 500000 :
	number of cases registered : 6
	all_max from 6.0 to 120.0
	most advanced time : 19
	current best min : 2
at count 600000 :
	number of cases registered : 11
	all_

at count 2200000 :
	number of cases registered : 8
	all_max from 6.0 to 91.0
	most advanced time : 19
	current best min : 2

For blueprint 20 , at start, min is : 1
at count 100000 :
	number of cases registered : 10
	all_max from 6.0 to 91.0
	most advanced time : 20
	current best min : 3

For blueprint 21 , at start, min is : 3

For blueprint 22 , at start, min is : 7
at count 100000 :
	number of cases registered : 10
	all_max from 15.0 to 78.0
	most advanced time : 18
	current best min : 8

For blueprint 23 , at start, min is : 0

For blueprint 24 , at start, min is : 11
at count 100000 :
	number of cases registered : 12
	all_max from 15.0 to 171.0
	most advanced time : 18
	current best min : 11
at count 200000 :
	number of cases registered : 15
	all_max from 15.0 to 171.0
	most advanced time : 19
	current best min : 13
at count 300000 :
	number of cases registered : 8
	all_max from 28.0 to 171.0
	most advanced time : 16
	current best min : 13
at count 400000 :
	number of cases regist

In [127]:
blueprints = get_blueprints(test_input)

In [129]:
blueprints = get_blueprints(my_input)

In [130]:
TMAX = 32

index_blueprint = []
result_blueprint = []

for ib in range(min(len(blueprints),3)) :

    b = blueprints[ib]


    all_R = [[1,0,0,0]]
    all_M = [[0,0,0,0]]
    all_T = [0]
    all_max = [max_collect(all_R[0],all_M[0],b,all_T[0],TMAX)]

    Overall_Min = max(min_collect(all_R[0],all_M[0],b,all_T[0],TMAX), 
                      get_one_path_shortest_time(b,TMAX),
                      get_one_path_most_build(b,TMAX))
    # if one_max < Overall_Min : then suppress
    print("\nFor blueprint",ib,", at start, min is :",Overall_Min)

    count = 0
    while len(all_R) > 0 :

        R = all_R.pop(-1)
        M = all_M.pop(-1)
        T = all_T.pop(-1)
    #     mi = all_min.pop(-1)
        ma = all_max.pop(-1)

        for iR in range(4) :

            TR = times_to_build_Rs[iR](R,M,b)

            if TR == math.inf :
                continue

            if T+TR >= TMAX :
                # only gather and stop
                Rn, Mn = update_gather_till_end(R,M,b,T,TMAX)
    #             print("\t",Mn[3])
                Overall_Min = max(Overall_Min,Mn[3])
                # keep only those that might do better
                keep = [all_max[i]>Overall_Min for i in range(len(all_max))]
                all_R = update_list(all_R,keep)
                all_M = update_list(all_M,keep)
                all_T = update_list(all_T,keep)
                all_max = update_list(all_max,keep)

                continue

            Rn, Mn = update_gather_and_build(R,M,b,iR,TR)
            # update_gather_and_build(R,M,b,iRbuilt,Ttobuilt)
            Tn = T + TR
            man = max_collect(Rn,Mn,b,Tn,TMAX)
            if man > Overall_Min :
                if Rn in all_R :
                    all_idx = [i for i in range(len(all_R)) if all_R[i] == Rn and all_M[i] == Mn]
                    if len(all_idx) > 0 :
                        if all_T[all_idx[0]] > Tn :
                            all_T[all_idx[0]] = Tn
                        continue
                all_R.append(Rn)
                all_M.append(Mn)
                all_T.append(Tn)
                all_max.append(man)

            if min(Mn) < 0:
                print("Problem : ", iR, TR)
                print("\t",R,M,T)
                print("\t",Rn,Mn,Tn)



        count += 1
        if count % 100000 == 0 :
            print("at count",count,":")
            print("\tnumber of cases registered :" ,len(all_R))
            print("\tall_max from",min(all_max),"to",max(all_max))
            print("\tmost advanced time :",max(all_T))
            print("\tcurrent best min :",Overall_Min)
            
    
    index_blueprint.append(ib+1)
    result_blueprint.append(Overall_Min)
    print("FOr blueprint",ib,", at the end, opened",Overall_Min,"geodes.\n")
    
# print(sum([index_blueprint[i]*result_blueprint[i] for i in range(len(result_blueprint))]))

print(result_blueprint[0]*result_blueprint[1]*result_blueprint[2])


For blueprint 0 , at start, min is : 17
at count 100000 :
	number of cases registered : 14
	all_max from 24.0 to 300.0
	most advanced time : 26
	current best min : 23
at count 200000 :
	number of cases registered : 18
	all_max from 25.0 to 300.0
	most advanced time : 27
	current best min : 23
at count 300000 :
	number of cases registered : 11
	all_max from 28.0 to 300.0
	most advanced time : 24
	current best min : 24
at count 400000 :
	number of cases registered : 15
	all_max from 28.0 to 300.0
	most advanced time : 25
	current best min : 24
at count 500000 :
	number of cases registered : 15
	all_max from 28.0 to 300.0
	most advanced time : 25
	current best min : 24
at count 600000 :
	number of cases registered : 18
	all_max from 28.0 to 231.0
	most advanced time : 26
	current best min : 25
at count 700000 :
	number of cases registered : 19
	all_max from 30.0 to 231.0
	most advanced time : 25
	current best min : 29
at count 800000 :
	number of cases registered : 13
	all_max from 37.0 

at count 3100000 :
	number of cases registered : 12
	all_max from 21.0 to 171.0
	most advanced time : 25
	current best min : 16
at count 3200000 :
	number of cases registered : 6
	all_max from 36.0 to 171.0
	most advanced time : 23
	current best min : 16
at count 3300000 :
	number of cases registered : 11
	all_max from 21.0 to 136.0
	most advanced time : 25
	current best min : 16
FOr blueprint 2 , at the end, opened 16 geodes.

10672
